In [ ]:
import keras
from keras.saving import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json

In [ ]:
#setting the start and end date to look at
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [ ]:
#initializing quant book, allows us to access their API data
qb = QuantBook()
symbol = qb.AddCrypto("BTCUSD", Resolution.Daily).Symbol
history = qb.History(symbol, start, end).loc[symbol]
history.head() #previews first 5 rows

In [ ]:
#isolating the data we want and excluding the rest
daily_pct_change = history[["open", "high", "low", "close", "volume"]].pct_change().dropna()
df = daily_pct_change
df.head()

In [ ]:
# removes all values of infinity and replaces with the highest finite value
# essentially cleans the data
indexes = df[((df.volume == float("inf")))].index
for i in indexes:
    df.at[i, "volume"] = max(df.volume.drop(indexes))

In [ ]:
#using the past 30 days to inform current days decision
n_steps = 30 # Step size (data per step)
features = [] # inputs
labels = [] # outputs
for i in range(len(df)-n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df['close'].iloc[i+n_steps] >= 0:
        # UP
        label = 1
    else:
        # DOWN
        label = 0
    labels.append(label)

In [ ]:
features = np.array(features) #convert to desired input type
labels = np.array(labels)

In [ ]:
train_length = int(len(features) * 0.7) # split the data for training and testing
X_train = features[:train_length]
X_test = features[train_length:]
y_train = labels[:train_length]
y_test = labels[train_length:]

In [ ]:
# checks the number of up vs down days in training data, should be relatively balanced
sum(y_train)/len(y_train)

In [ ]:
# use second part of data for testing instead
test_length = int(len(features) * 0.3)
X_train = features[test_length:]
X_test = features[:test_length]
y_train = labels[test_length:]
y_test = labels[:test_length]

In [1]:
sum(y_train) / len(y_train)

NameError: name 'y_train' is not defined

In [ ]:
#                   features, shape, relu gets rid of negative
model = Sequential([Dense(30, input_shape=X_train[0].shape, activation='relu'),
                    Dense(20, activation='relu'),
                    Flatten(),
                    Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy', 'mse'])

In [ ]:
model.fit(X_train, y_train, epochs=10) #epochs is how many times it loops through the data

In [ ]:
y_hat = model.predict(X_test) #now test the model

In [ ]:
results = pd.DataFrame({'y': y_test.flatten(), 'y_hat': y_hat.flatten()})

In [ ]:
results.plot(title='Model Performance: predicted vs actual %change in closing price', figsize=(17, 7))

In [ ]:
pred_train = model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))

pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

In [ ]:
model_str = json.dumps(serialize_keras_object(model)) #turn model to a json object

In [ ]:
model_key = 'bitcoin_price_predictor' #give model a name

In [ ]:
qb.ObjectStore.Save(model_key, model_str) #save the model

In [ ]:
if qb.ObjectStore.ContainsKey(model_key):
    model_str = qb.ObjectStore.Read(model_key)
    config = json.loads(model_str)['config']
    model = Sequential.from_config(config)

In [ ]:
testDate = datetime.now()

In [ ]:
df = qb.History(symbol, testDate - timedelta(40), testDate).loc[symbol]
df_change = df[["open", "high", "low", "close", "volume"]].pct_change().dropna()
model_input = []
for index, row in df_change.tail(30).iterrows():
    model_input.append(np.array(row))
model_input = np.array([model_input])
#print(model_input)

In [ ]:
print((model.predict(model_input)[0][0] - 0.5) * 2)
if round(model.predict(model_input)[0][0]) == 0:
    print("down")
else:
    print("up")